In [1]:
import os
import sys
sys.path.append("/project/stockAI/github/main/stockAI")
import pandas as pd
import stockAI as sai

In [2]:
##### 트레이더 하나 정의 -> 4개로 카피 -> 데이터셋 저장을 for문으로 ! 

# 1. Data Acquisition 

한국의 KOSPI 데이터 중에서 100개의 종목으로만 진행 

In [ ]:
lst_tickers = sai.get_tickers(markets=['KOSPI'], date=2016)
print(len(lst_tickers), lst_tickers[:5])

In [ ]:
raw_data = sai.load_data(date=['2016-01-01', '2021-12-31'], tickers=lst_tickers[:100])
print(raw_data.shape)
raw_data.head()

# 2. Data Preprocessing 

In [ ]:
check_index = ['MA5', 'MA20', 'MA60','MA120', 
             'trading_value','next_change',
             'MFI','ADI','OBV','CMF','FI','EOM_EMV','VPT','NVI','VMAP',
             'BHB','BLB','KCH','KCL','KCM','DCH','DCL','DCM','UI',
             'SMA','EMA','WMA','MACD','VIneg','VIpos','TRIX','MI','CCI','DPO','KST','Ichimoku','ParabolicSAR','STC',
             'RSI','SRSI','TSI','UO','SR','WR','AO','KAMA','ROC','PPO','PVO']

check_df = sai.add_index(data=raw_data, index_list=check_index)
check_df

### 2) scaling 

In [ ]:
scaled_minmax = sai.scaling(data=check_df, scaler_type="minmax", window_size=None)
scaled_minmax

In [ ]:
scaled_standard = sai.scaling(data=check_df, scaler_type="standard", window_size=None)
scaled_standard

In [ ]:
scaled_robust = sai.scaling(data=check_df, scaler_type="robust", window_size=None)
scaled_robust

In [ ]:
scaled_div_close = sai.scaling(data=check_df, scaler_type="div-close", window_size=None)
scaled_div_close

# 3. Trader Definition

In [ ]:
lst_trader = []

In [ ]:
from lightgbm import LGBMClassifier

# conditional_buyer: Object that determines acquisition based on data filtering conditions 
b1_pr = sai.conditional_buyer()

def sampling3(df): # 조건 함수 생성 
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) # Remove exceptions that exceed upper and lower limits
    condition2 = df.D0_trading_value >= 1000000000 # condition 1: Transaction amount of more than 1 billion won 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) # condition 2: Today's stock price change rate is more than 5%
    condition = condition1 & condition2 & condition3
    return condition

b1_pr.condition = sampling3  # Define the condition function directly (sampling1) and store it in the condition property 


# machinelearning_buyer: Object that determines acquisition by machine learning model
b2_pr = sai.machinelearning_buyer()

# Save user-defined models to algorithm properties
scale_pos_weight = round(72/28 , 2)
params = {  'random_state' : 42,
            'scale_pos_weight' : scale_pos_weight,
            'learning_rate' : 0.1, 
            'num_iterations' : 1000,
            'max_depth' : 4,
            'n_jobs' : 30,
            'boost_from_average' : False,
            'objective' : 'binary' }

b2_pr.algorithm =  LGBMClassifier( **params )


# SubSeller: Object that determines selling all of the following days
sell_all = sai.SubSeller() 


# Trader Object   
trader = sai.Trader()
trader.label = 'class&0.02' # Set the Trader dependent variable (do not set if it is regression analysis) 
trader.buyer = sai.Buyer([b1_pr, b2_pr]) # [ conditional buyer, machinelearning buyer ] 
trader.seller = sai.Seller(sell_all)

# 4. Trader(Model) Fitting & Evauation

### 1) Save Dataset to Traders

In [ ]:
trader_name = ["trader_minmax", "trader_standard", "trader_robust", "trader_div-close"]
for name in trader_name: 
    trader.name = name
    lst_trader.append(trader)

In [ ]:
lst_trader = []
lst_dataset_scaled = [[train_data_minmax, test_data_minmax], 
                      [train_data_standard, test_data_standard],
                     [train_data_robust, test_data_robust],
                     [train_data_div_close, test_data_div_close]]

for i in range(4): 
    lst_trader_one = [lst_trader[i]]
    sai.save_dataset(lst_trader_one, train_data, lst_dataset_scaled[i][0], lst_dataset_scaled[i][1])
    lst_trader.append(trader)

### 2) Model fitting

In [ ]:
sai.trader_train(lst_trader) 